<a href="https://colab.research.google.com/github/koya-jp/AA-google-colab-kohya/blob/master/Diffusers_S2D2__Korean_bikini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Diffusers ライブラリを用いて、画像を生成するスクリプト。**

[Github](https://github.com/koya-jp/AA-google-colab-kohya/blob/master/Diffusers_S2D2.ipynb)

In [1]:
#@title Driveに接続, ライブラリの追加 { display-mode: "form" }

# Driveに接続
from google.colab import drive
drive.mount('/content/drive')

# ライブラリの追加
# !pip install diffusers==0.12.1
# diffusers[torch] 以外の のインストール
!pip install --upgrade diffusers==0.17.1 transformers accelerate scipy ftfy safetensors txt2img >/dev/null 2>&1
!pip install k-diffusion >/dev/null 2>&1


Mounted at /content/drive


In [2]:
#@title 「S2D2」のリポジトリを複製し、起動するのに必要な要件をインストール { display-mode: "form" }

%cd /content
!git clone https://github.com/keisuke-okb/S2D2

%cd ./S2D2
!git pull
!touch __init__.py
!pip install -r requirements.txt >/dev/null 2>&1


/content
Cloning into 'S2D2'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 70 (delta 37), reused 17 (delta 17), pack-reused 29
Unpacking objects: 100% (70/70), 23.95 KiB | 817.00 KiB/s, done.
/content/S2D2
Already up to date.


In [3]:
#@title S2D2の準備、embeddingsを読み込む関数「load_embeddings」の定義 { display-mode: "form" }

import s2d2
import os
import random
import datetime
import torch
from PIL import Image

import diffusers
from diffusers import (StableDiffusionPipeline,StableDiffusionImg2ImgPipeline)
from diffusers.utils import numpy_to_pil
from diffusers.pipelines.stable_diffusion import StableDiffusionPipelineOutput

class StableDiffusionImageGeneratorMod(s2d2.StableDiffusionImageGenerator):
  def __init__(self, sd_safetensor_path: str, device: str="cuda", dtype: torch.dtype=torch.float16):
    self.device = torch.device(device)
    self.pipe = StableDiffusionPipeline.from_pretrained(
      sd_safetensor_path,
      torch_dtype=dtype,
      custom_pipeline="lpw_stable_diffusion"
    ).to(device)
    self.pipe_i2i = StableDiffusionImg2ImgPipeline.from_pretrained(
      sd_safetensor_path,
      torch_dtype=dtype,
      custom_pipeline="lpw_stable_diffusion"
    ).to(device)
    self.pipe.safety_checker = None
    self.pipe_i2i.safety_checker = None
    return

  def load_embeddings(self, safetensors_path: str, fileName: str, token: str):
    if token != "":
      self.pipe.load_textual_inversion(safetensors_path, weight_name=fileName, token=token)
      self.pipe_i2i.load_textual_inversion(safetensors_path, weight_name=fileName, token=token)
    else:
      self.pipe.load_textual_inversion(safetensors_path, weight_name=fileName)
      self.pipe_i2i.load_textual_inversion(safetensors_path, weight_name=fileName)
    return

  %cd /content/

/content


In [ ]:
# @title Model、LoRA の読み込み　 { display-mode: "form" }

#モデル
model_path = "emilianJR/majicMIX_realistic_v6" #@param ["/content/drive/MyDrive/StableDiffusion/Model/chilledremix","runwayml/stable-diffusion-v1-5", "emilianJR/majicMIX_realistic_v6"]

generator = StableDiffusionImageGeneratorMod(
  model_path,
  device="cuda",
)

LoRA_USE = True #@param {type:"boolean"}
if LoRA_USE == True:
  LoRA="/content/drive/MyDrive/StableDiffusion/Lora/flat2.safetensors" #@param {type:"string"}
  LoRA_alpha = -1.0 #@param {type:"number"}
  generator.load_lora(LoRA, alpha=LoRA_alpha)

LoRA_USE_2= True #@param {type:"boolean"}
if LoRA_USE_2== True:
  LoRA_2="/content/drive/MyDrive/StableDiffusion/Lora/japaneseDollLikeness_v15.safetensors" #@param {type:"string"}
  LoRA_alpha_2 = 0.6 #@param {type:"number"}
  generator.load_lora(LoRA_2, alpha=LoRA_alpha_2)


unet/diffusion_pytorch_model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [ ]:
# @title Embeddings を設定。

#@markdown **Embeddings 1**
embeddings_USE = True #@param {type:"boolean"}
if embeddings_USE == True:
  safetensors_path = "/content/drive/MyDrive/StableDiffusion/embeddings/EasyNegative.safetensors" #@param {type:"string"}
  file_name = "EasyNegative.safetensors" #@param {type:"string"}
  #@markdown ***トリガーワード（EasyNegativeなど）***
  token = "EasyNegative" #@param {type:"string"}
  generator.load_embeddings(safetensors_path, file_name, token)

#@markdown **Embeddings 2**
embeddings_USE_2 = True #@param {type:"boolean"}
if embeddings_USE_2 == True:
  safetensors_path_2 = "/content/drive/MyDrive/StableDiffusion/embeddings/ulzzang-6500-v1.1.pt" #@param {type:"string"}
  file_name_2 = "ulzzang-6500-v1.1.pt" #@param {type:"string"}
  token_2 = "ulzzang-6500-v1.1.pt" #@param {type:"string"}
  generator.load_embeddings(safetensors_path_2, file_name_2, token_2)

#@markdown **Embeddings 3**
embeddings_USE = True #@param {type:"boolean"}
if embeddings_USE == True:
  safetensors_path_3 = "/content/drive/MyDrive/StableDiffusion/embeddings/pureerosface_v1.pt" #@param {type:"string"}
  file_name_3 = "pureerosface_v1.pt" #@param {type:"string"}
  token_3 = "pureerosface_v1" #@param {type:"string"}
  generator.load_embeddings(safetensors_path_3, file_name_3, token_3)

In [ ]:
# @title 画像を生成  （content/drive/MyDrive/txt2img_output-タイムスタンプ に保存。）

# 必要なモジュールやライブラリをインポートする
import os
import shutil
from google.colab import files
from PIL.PngImagePlugin import PngInfo

# ファイル名に使う日付と時刻のフォーマットを定義する
file_format = "%Y%m%d_%H%M%S"
i=0

# 現在の日本時間を取得
jst_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
#txt2img出力画像の保存先
#@markdown 出力画像を保存するフォルダ
save_path = f"/content/drive/MyDrive/txt2img_output-{jst_now.strftime(file_format)}"
os.makedirs(save_path, exist_ok=True)


#@markdown ポジティブプロンプト
prompt = "Beautiful girl in micro bikini,(masterpiece),((super detailed)),(expressionless)), realistic8K UHD,(best quality:1. 2), high definition, exquisite details, fine texture, high detail, fine beautiful delicate eyes, perfectly proportioned face, light particles, distinct_ image, high resolution, high quality textures and shadows, realistic and beautiful face with big eyes, blush, glossy lips and perfect proportions, depth of field, lens flare, ray tracing, perspective, prominent nose, slender face, perfectly toned body, (thin waist:1. 3), medium chest, (glossy (glossy skin), (pureerosface_v1:0.5), (ulzzang-6500-v1.1:0.5), (sweat is visible on her body due to her strenuous exercise, her clothes are soaked and see through: 1. 2)), braun short hair, sitting, (spreading legs:1.5)), ((low angle)), camel toes,(exposed bikini:1.4)BREAK (smiling, embarrassed), nsfw ,in pool" #@param {type:"string"}
#@markdown ネガティブプロンプト
negative = "EasyNegative, painting, sketch, (worst picture quality:2), (low quality:2), (normal quality:2), bad feet, lowres, ((monochrome)), skin blemishes, acne, skin blemishes, bad anatomy, bad hands, text, error, missing fingers, extra fingers, few digits, crop, worst picture quality, jpeg image, signature, watermark, Username, blurred, bad feet, (mutation, deformity:1. 3), extra limbs, fused fingers, long neck, crossed eyes, very low resolution, bad proportions,(ng_deepnegative_v1_75t),(negative_hand-neg)" #@param {type:"string"}
#@markdown スケジューラ(サンプラー)
scheduler="dpm++_2m_karras" #@param ["unipc","euler_a","euler","ddim","ddpm","deis","dpm2","dpm++_2s","dpm++_2m","dpm++_2m_karras","dpm++_sde","dpm++_sde_karras","heun","heun_karras","lms","lms_karras","pndm","dpm++_2m_karras"]
#@markdown ステップ数
steps = 30 #@param {type:"integer"}
#@markdown 出力枚数
batch_count = 50 #@param {type: "integer"}
#@markdown 画像サイズ
img_width = "512" # @param [512, 768, 1024, 1536]
img_height = "768" # @param [512, 768, 1024, 1536]
#@markdown CFG
CFG = 7 #@param {type: "number"}
#@markdown シード（-1の時はランダム）
seed = -1 #@param {type: "integer"}
if seed is None or seed == -1:
  inputSeed = random.randint(0, 2147483647)
else:
  valueSeed = seed

#@markdown Hires.fix の有効化
hires_fix = True #@param {type: "boolean"}
enhance_steps = 2 if hires_fix else 1
#@markdown 解像度倍率(乗算後、最も近い8の倍数のサイズとなる)
upscaling_ratio = 2 #@param {type: "number"}
#@markdown アップスケールステップ数
up_steps = 20 #@param {type: "integer"}
#@markdown denoising strength（小さいほど元画像を尊重） （0.5～0.7）が推奨
denoising_strength = 0.60 #@param {type: "number"}


for i in range(batch_count):
  if seed is None or seed == -1:valueSeed = inputSeed + i
  image = generator.diffusion_enhance(
    prompt,
    negative,
    scheduler_name=scheduler,
    num_inference_steps=steps,
    num_inference_steps_enhance=up_steps,
    guidance_scale=CFG,
    width=img_width,
    height=img_height,
    seed=valueSeed,
    upscale_target="latent",
    interpolate_mode="bicubic",
    antialias=True,
    upscale_by=upscaling_ratio,
    enhance_steps=enhance_steps,
    denoising_strength=denoising_strength,
    output_type="pil",
    decode_factor=0.15,
    decode_factor_final=0.18215
  )

  # 現在の日本時間を取得
  jst_now = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
  #出力する画像の名前を生成する
  file_name = (jst_now.strftime(file_format)+ "_" + str(valueSeed))
  image_name = file_name + f".png"

  #画像を保存する
  save_location = os.path.join(save_path, image_name)
  #@markdown **メタデータの書き込み**
  save_metadata = True #@param {type: "boolean"}
  if save_metadata:
    metadata = PngInfo()
    metadata.add_text("prompt",(prompt))
    metadata.add_text("negative",(negative))
    metadata.add_text("scheduler",(scheduler))
    metadata.add_text("steps",(str(steps)))
    metadata.add_text("CFG",(str(CFG)))
    metadata.add_text("width",(str(img_width)))
    metadata.add_text("height",(str(img_height)))
    metadata.add_text("seed",str((valueSeed)))
  if hires_fix:
    metadata.add_text("upscaling ratio",str((upscaling_ratio)))
    metadata.add_text("up steps",str((up_steps)))
    metadata.add_text("denoising strength",str((denoising_strength)))
    image.save(save_location, pnginfo=metadata)

  else:
    image.save(save_location)

# @markdown **ダウンロード済のフォルダの移動先フォルダを作る**
end_downloading_dir = "/content/drive/MyDrive/End-downloading" # @param {type:"string"}
os.makedirs(end_downloading_dir, exist_ok=True)

# ファイル名を生成する
file_name = save_path.split("/")[-1]
# フォルダをzip圧縮する
shutil.make_archive(file_name, 'zip', save_path)
print(file_name)

# Zipをダウンロード
files.download(f'/content/{file_name}.zip')

# 終わったらフォルダをend_downloading_dirに場所に移動する
shutil.move(os.path.join("/content/drive/MyDrive", save_path), end_downloading_dir)


In [ ]:
# @title 画像のメタデータを出力

import sys
from PIL import Image
from PIL.PngImagePlugin import PngInfo

#@markdown **保存した画像のパス**
file_path = "/content/drive/MyDrive/txt2img_output-20230708_013952/20230708_014048_825314091.png" #@param {type: "string"}
img = Image.open(file_path)

print("Prompt: ",img.text["prompt"])
print("Negative Prompt: ",img.text["negative"])
print("Scheduler: ", img.text["scheduler"])
print("Steps: ",img.text["steps"])
print("CFG: ",img.text["CFG"])
print("Width: ",img.text["width"])
print("Height: ",img.text["height"])
print("Seed: ",img.text["seed"])

try:
  print("Upscaling ratio; ",img.text["upscaling ratio"])
  print("Up steps: ",img.text["up steps"])
  print("Denoising strength: ",img.text["denoising strength"])
except:
  print("Hires.fix was OFF.")

# %%writefile output.txt